https://huggingface.co/NLP-LTU/bertweet-large-sexism-detector?

In [53]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer,pipeline
import torch
import pandas as pd
import json

In [54]:
model = AutoModelForSequenceClassification.from_pretrained('NLP-LTU/bertweet-large-sexism-detector')
tokenizer = AutoTokenizer.from_pretrained('NLP-LTU/bertweet-large-sexism-detector') 
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
prediction=classifier("I like cats")

In [55]:

print(prediction)

[{'label': 'not sexist', 'score': 0.9991477727890015}]


In [56]:
class SexismEvaluator():
    def __init__(self, fileTweet):
        self.model = AutoModelForSequenceClassification.from_pretrained('NLP-LTU/bertweet-large-sexism-detector')
        self.tokenizer = AutoTokenizer.from_pretrained('NLP-LTU/bertweet-large-sexism-detector') 
        self.classifier = pipeline("text-classification", model=self.model, tokenizer=self.tokenizer)
        self.fileTweet = pd.read_csv(fileTweet, sep=",")

    def evaluate(self, fileResult):
        self.rateTweets(fileResult)

    def rateTweets(self, fileResult):
        datalist = []
        for idx, tweet in self.fileTweet.iterrows():
            date = tweet.loc["DateTime"]
            sentence = tweet.loc["Text"]
            prediction = self.classifier(sentence)
            score = prediction[0]["score"]
            sexist = prediction[0]["label"]
            data=[
                date,
                sentence,
                sexist,
                score
            ]
            datalist.append(data)
        tweet_df = pd.DataFrame(datalist, columns=["date", "sentence", "label", "score"])
        tweet_df.to_csv(f"{fileResult}.csv", sep=",")


In [57]:
fileResult = "evaluations"
fileTweet = "2022-08-01_2022-08-31_100.csv"
evaluator = SexismEvaluator(fileTweet)
evaluator.evaluate(fileResult)